<a href="https://colab.research.google.com/github/liveleisurely/NLP/blob/main/Faiss%EB%A5%BC%20%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC%20SBERT%20%EB%B9%A0%EB%A5%B4%EA%B2%8C%20%EA%B2%80%EC%83%89%ED%95%98%EA%B8%B0%20%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-gpu
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=75b601ace5467183d9db81ad51b6b3e4795ea932632874329d40b76ab23d2e40
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54

In [1]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv", filename="abcnews-date-text.csv")

df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()

In [3]:
data[:5]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

In [4]:
print('총 샘플의 개수 :', len(data))

총 샘플의 개수 : 1082168


In [6]:
import random

In [7]:
# 데이터가 커서 encode에 어려움 => 만개로 줄여서 실습
sample = random.sample(data, 10000)
sample

['dakar rally champion toby price set to win finke desert race',
 'gas industry opposes call to reserve local supplies',
 'fitzroy group shares in reef rescue funds',
 '8m for anu scientific research',
 'lawyer calls for release of businessman matthew ng',
 'sa opposition takes keen interest in nsw lakes',
 'nato forces investigate death of bbc journalist',
 'country hour debate over the power of the big two',
 'indian women beat australia by five wickets to win final odi',
 "peter greste 'guided by the devil' to defame egypt",
 'adelaide think tank full of inspiration for growth',
 'gas leak sparks tafe evacuation',
 'wallabies not playing for world cup spots against usa',
 'man died due to lack of hospital treatment lawyer',
 'philae lander robot on comet passing sun',
 'ralf fastest in barcelona testing',
 'south highlands coal seam gas warning',
 'sale of western power debt to superannuation funds considered',
 '4 charged over body in burnt car',
 'webber dominates malaysian practi

In [8]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(sample)
print('임베딩 된 벡터 수 :', len(encoded_data))

임베딩 된 벡터 수 : 10000


In [12]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(sample))))

faiss.write_index(index, 'abc_news')

In [10]:
def search(query):
   t = time.time()
   query_vector = model.encode([query])
   k = 5
   top_k = index.search(query_vector, k)
   print('total time: {}'.format(time.time() - t))
   return [data[_id] for _id in top_k[1].tolist()[0]]

In [13]:
query = str(input())
results = search(query)

print('results :')
for result in results:
   print('\t', result)

Underwater Forest Discovered
total time: 0.02543950080871582
results :
	 share market plunge with global uncertainty
	 ominami wins nagoya marathon
	 tasmanian shops will open on easter sunday
	 claims council costs making it hard to keep rates
	 french hit by galthie withdrawal
